In [1]:
import pandas as pd
from secrets_config import source_db_user, source_db_password, source_db_server_name, source_db_database_name
import jinja2 as j2 

# import libraries for sql 
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

In [2]:
import os 
import logging 


def extract_from_database(table_name, engine, path="extract_queries")->pd.DataFrame:
    """
    Builds models with a matching file name in the models_path folder. 
    - `table_name`: the name of the table (without .sql)
    - `path`: the path to the extract queries directory containing the sql files. defaults to `extract_queries`
    """
    logging.basicConfig(level=logging.INFO, format="[%(levelname)s][%(asctime)s]: %(message)s")
    
    if f"{table_name}.sql" in os.listdir(path):
        logging.info(f"Extracting table: {table_name}")
    
        # read sql contents into a variable 
        with open(f"{path}/{table_name}.sql") as f: 
            raw_sql = f.read()

        # parse sql using jinja 
        parsed_sql = j2.Template(raw_sql).render(source_table = table_name, engine=engine)
        # # execute parsed sql 
        df = pd.read_sql(sql=parsed_sql, con=engine)

        logging.info(f"Successfully extracted table: {table_name}, rows extracted: {len(df)}")
        return df 
    else: 
        logging.error(f"Could not find table: {table_name}")